EDA

In [4]:
#Importamos las librerias a utilizar

import pandas as pd
import json
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

In [5]:
#Cargamos el dataset a utilizar

data=pd.read_csv("Datasets\movies_dataset.csv")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18732\3953064996.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("Datasets\movies_dataset.csv")


In [6]:
#Visualizamos el dataset

data.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [7]:
#Visualizamos las columnas que contiene el dataset

data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [8]:
#Seleccionamos las columnas que nos interesan para hacer las operaciones planteadas, en este caso un modelo de recomendacion y dejamos el dataset con esas columnas nada mas

columnas_deseadas = ['id',"title",'genres', 'overview', "release_date", "spoken_languages","popularity"]
data = data[columnas_deseadas]

In [9]:
#Verificamos cuantos duplicados tenemos con el id de cada pelicula

data['id'].duplicated().sum()

30

In [10]:
#Eliminamos los duplicados en la columna "id"

data.drop_duplicates(subset='id', inplace=True)

In [11]:
#Consultamos cuntos nulos tenemos en la columna "genres"

data["genres"].isnull().sum()

0

In [12]:
#Consultamos cuntos nulos tenemos en la columna "spoken_languages"

data["spoken_languages"].isnull().sum()

6

In [13]:
#Consultamos cuntos nulos tenemos en la columna "release_date"

data["release_date"].isnull().sum()

87

In [14]:
#Consultamos cuntos nulos tenemos en la columna "popularity"

data["popularity"].isnull().sum()

5

In [15]:
#Consultamos cuntos nulos tenemos en la columna "overview"

data["overview"].isnull().sum()

954

In [16]:
#Considerando que la fecha de lanzamiento es importante, eliminamos las filas donde 'release_date' sea nulo

data = data.dropna(subset=['release_date']).reset_index(drop=True)

In [17]:
#Creamos una nueva columna con el año de lanzamiento de cada pelicula

data["release_year"] = pd.to_datetime(data["release_date"], errors="coerce").dt.year

In [18]:
#Consultamos cuntos nulos tenemos en la columna "release_year"

data["release_year"].isnull().sum()

3

In [19]:
#Eliminamos las filas donde 'release_year' sea nulo

data = data.dropna(subset=['release_year']).reset_index(drop=True)

In [20]:
#Transformamos el tipo de dato a entero de los datos de la columna 'release_year'

data['release_year'] = data['release_year'].astype(int)

In [21]:
#Eliminamos la columna 'release_date'

data.drop('release_date', axis=1, inplace=True)

In [22]:
#Visualizamos el dataset

data.head(3)

,id,title,genres,overview,spoken_languages,popularity,release_year
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'iso_639_1': 'en', 'name': 'English'}]",21.946943,1995
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",17.015539,1995
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,"[{'iso_639_1': 'en', 'name': 'English'}]",11.7129,1995


In [23]:
#definimos esta funcion para exceptuar los errores con los valores faltantes
def safe_json_loads(value):
    try:
        return json.loads(value)
    except: #(json.JSONDecodeError, TypeError):
        return {}

In [24]:
#Queremos dejar una lista con los generos de cada paelicula, ya que tenemos es un diccionario de listas
#Creamos un dataframe de la columna a tratar

genres= pd.DataFrame(data["genres"])

#Reseteamos el indice

genres = genres.reset_index(drop=True)

#Reemplazamos comillas dobles por simples

genres['genres'] = genres['genres'].str.replace("'", '"')

#Rellenamos nulos con un diccionario vacio

genres['genres'] = genres['genres'].fillna('{}')

#Convertimos la columna que contiene cadenas JSON en una columna que contiene listas de nombres correspondientes a cada elemento del JSON.

genres['genres'] = genres['genres'].apply(lambda x: [d['name'] for d in safe_json_loads(x)])

#Eliminamos la columna 'genres' del dataset original

data = data.drop('genres', axis=1)

#Concatenamos el dataset genres al dataset original y ya queda en forma de lista

data = pd.concat([data, genres], axis=1)

In [25]:
#Queremos dejar una lista con los generos de cada paelicula, ya que tenemos es un diccionario de listas
#Creamos un dataframe de la columna a tratar

spoken_languages= pd.DataFrame(data["spoken_languages"])

#Reseteamos el indice

spoken_languages = spoken_languages.reset_index(drop=True)

#Reemplazamos comillas dobles por simples

spoken_languages['spoken_languages'] = spoken_languages['spoken_languages'].str.replace("'", '"')

#Rellenamos nulos con un diccionario vacio

spoken_languages['spoken_languages'] = spoken_languages['spoken_languages'].fillna('{}')

#Convertimos la columna que contiene cadenas JSON en una columna que contiene listas de nombres correspondientes a cada elemento del JSON.

spoken_languages['spoken_languages'] = spoken_languages['spoken_languages'].apply(lambda x: [d['name'] for d in safe_json_loads(x)])

#Eliminamos la columna 'spoken_languages' del dataset original

data = data.drop('spoken_languages', axis=1)

#Concatenamos el dataset spoken_languages al dataset original y ya queda en forma de lista

data = pd.concat([data, spoken_languages], axis=1)

In [26]:
data

,id,title,overview,popularity,release_year,genres,spoken_languages
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,"[Animation, Comedy, Family]",[English]
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995,"[Adventure, Fantasy, Family]","[English, Français]"
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995,"[Romance, Comedy]",[English]
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995,"[Comedy, Drama, Romance]",[English]
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995,[Comedy],[English]
...,...,...,...,...,...,...,...
45341,30840,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,1991,"[Drama, Action, Romance]",[English]
45342,111109,Century of Birthing,An artist struggles to finish his work while a...,0.178241,2011,[Drama],[]
45343,67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003,"[Action, Drama, Thriller]",[English]
45344,227506,Satan Triumphant,"In a small town live two brothers, one a minis...",0.003503,1917,[],[]


In [27]:
#Eliminamos filas que contengan una lista vacía en alguna columna
data = data[data.apply(lambda row: not any(isinstance(val, list) and len(val) == 0 for val in row), axis=1)]

In [28]:
#Eliminamos espacios al principio y final de los datos de la columna "title" y "overview", "overview" lo pasamos a minuscula todos los caracteres

data["title"] = data["title"].str.strip()
data["overview"] = data["overview"].str.strip().str.lower()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18732\1376088848.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["title"] = data["title"].str.strip()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18732\1376088848.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["overview"] = data["overview"].str.strip().str.lower()


In [29]:
#Definimos la funcion para limpiar el texto de la columna overview

def clean_text(text):
    #Verificamos si el valor es una cadena de texto

    if isinstance(text, str):  

        #Eliminamos caracteres especiales y puntuación

        text = re.sub(r'[^\w\s]', '', text)

        #Convertimos a minúsculas
        
        text = text.lower()
    return text


In [30]:
#Aplicamos la funcion para limpiar texto a 'overview'

data['overview'] = data['overview'].apply(clean_text)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18732\3932186009.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['overview'] = data['overview'].apply(clean_text)


In [31]:
#Visualizamos los duplicados en la columna "title"

data.duplicated(subset=["title"]).value_counts()

False    37600
True      2691
Name: count, dtype: int64

In [32]:
#Para fines del modelo no nos interesan titulos duplicados, ya que hay titulos repetidos pero de diferentes años, y para hacer el dataset mas pequeño omitimos esas repeticiones y nos quedamos con la primera aparicion

data.drop_duplicates(subset=["title"], keep='first', inplace=True)
data.reset_index(drop=True, inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18732\2597211142.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(subset=["title"], keep='first', inplace=True)


In [33]:
#Descargar recursos necesarios para NLTK

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
#Creamos objeto de lematización

lemmatizer = WordNetLemmatizer()

In [35]:
#Obtenemos lista de stopwords en inglés

stopwords_list = set(stopwords.words('english'))

In [36]:
#Definimos una funcion para trtar cadenas de texto

def preprocess_text(text):

    #Verificamos si el valor es una cadena de texto

    if isinstance(text, str):

        #Tokenización

        tokens = word_tokenize(text)
        
        #Eliminación de puntuación y palabras vacías

        tokens = [token.lower() for token in tokens if token.lower() not in stopwords_list and token.lower() not in string.punctuation]
        
        # Lematización

        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Unimos tokens en una cadena de texto nuevamente

        processed_text = ' '.join(tokens)
        return processed_text
    else:

        # Devolvemos una cadena vacía para valores no válidos

        return ''  


In [37]:
# Aplicamos la función de preprocesamiento a la columna 'overview' en el dataFrame

data['overview'] = data['overview'].apply(preprocess_text)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18732\485640941.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['overview'] = data['overview'].apply(preprocess_text)


In [38]:
data.to_csv("Datasets\data_con_EDA.csv", index=False)


In [39]:
data 

,id,title,overview,popularity,release_year,genres,spoken_languages
0,862,Toy Story,led woody andys toy live happily room andys bi...,21.946943,1995,"[Animation, Comedy, Family]",[English]
1,8844,Jumanji,sibling judy peter discover enchanted board ga...,17.015539,1995,"[Adventure, Fantasy, Family]","[English, Français]"
2,15602,Grumpier Old Men,family wedding reignites ancient feud nextdoor...,11.7129,1995,"[Romance, Comedy]",[English]
3,31357,Waiting to Exhale,cheated mistreated stepped woman holding breat...,3.859495,1995,"[Comedy, Drama, Romance]",[English]
4,11862,Father of the Bride Part II,george bank recovered daughter wedding receive...,8.387519,1995,[Comedy],[English]
...,...,...,...,...,...,...,...
37595,84419,House of Horrors,unsuccessful sculptor save madman named creepe...,0.222814,1946,"[Horror, Mystery, Thriller]",[English]
37596,390959,Shadow of the Blair Witch,truecrime documentary delve murder spree inspi...,0.076061,2000,"[Mystery, Horror]",[English]
37597,289923,The Burkittsville 7,film archivist revisits story rustin parr herm...,0.38645,2000,[Horror],[English]
37598,222848,Caged Heat 3000,year 3000 ad world dangerous woman banished re...,0.661558,1995,[Science Fiction],[English]
